In [2]:
import requests
import json
from getpass import getpass
import base64


In [3]:
# set credentials
ACCESS_TOKEN = getpass()

### repo content

In [12]:
# url for repo content
url = "https://api.github.com/repos/taniiishk/rock-paper-scissors-game/contents/"

payload = {}
headers = {
  'Accept': 'application/vnd.github+json',
  'X-GitHub-Api-Version': '2022-11-28',
  'Authorization': f'Bearer {ACCESS_TOKEN}'
}

In [13]:
response = requests.request("GET", url, headers=headers, data=payload)
if response.status_code == 200:
    print('Request successful')
else:
    raise Exception(f"Non-success status code: {response.status_code}")

Request successful


In [14]:
data = response.json()
data

[{'name': 'README.md',
  'path': 'README.md',
  'sha': 'd1b07bc9e30f12188a6be5575fd4a27114cbf874',
  'size': 3287,
  'url': 'https://api.github.com/repos/Taniiishk/Rock-Paper-Scissors-Game/contents/README.md?ref=main',
  'html_url': 'https://github.com/Taniiishk/Rock-Paper-Scissors-Game/blob/main/README.md',
  'git_url': 'https://api.github.com/repos/Taniiishk/Rock-Paper-Scissors-Game/git/blobs/d1b07bc9e30f12188a6be5575fd4a27114cbf874',
  'download_url': 'https://raw.githubusercontent.com/Taniiishk/Rock-Paper-Scissors-Game/main/README.md',
  'type': 'file',
  '_links': {'self': 'https://api.github.com/repos/Taniiishk/Rock-Paper-Scissors-Game/contents/README.md?ref=main',
   'git': 'https://api.github.com/repos/Taniiishk/Rock-Paper-Scissors-Game/git/blobs/d1b07bc9e30f12188a6be5575fd4a27114cbf874',
   'html': 'https://github.com/Taniiishk/Rock-Paper-Scissors-Game/blob/main/README.md'}},
 {'name': 'code_file.py',
  'path': 'code_file.py',
  'sha': 'ee15d676e980576746be8cbbe93779ddfd3c9187

### one file

In [18]:
url_file = "https://api.github.com/repos/taniiishk/rock-paper-scissors-game/contents/code_file.py"
response = requests.request("GET", url_file, headers=headers, data=payload)


In [22]:
data = response.json()
data

{'name': 'code_file.py',
 'path': 'code_file.py',
 'sha': 'ee15d676e980576746be8cbbe93779ddfd3c9187',
 'size': 2446,
 'url': 'https://api.github.com/repos/Taniiishk/Rock-Paper-Scissors-Game/contents/code_file.py?ref=main',
 'html_url': 'https://github.com/Taniiishk/Rock-Paper-Scissors-Game/blob/main/code_file.py',
 'git_url': 'https://api.github.com/repos/Taniiishk/Rock-Paper-Scissors-Game/git/blobs/ee15d676e980576746be8cbbe93779ddfd3c9187',
 'download_url': 'https://raw.githubusercontent.com/Taniiishk/Rock-Paper-Scissors-Game/main/code_file.py',
 'type': 'file',
 'content': 'aW1wb3J0IHRraW50ZXIgYXMgdGsKZnJvbSB0a2ludGVyIGltcG9ydCBQaG90\nb0ltYWdlCmltcG9ydCByYW5kb20KCnJvb3QgPSB0ay5UaygpCnJvb3QudGl0\nbGUoIlJvY2sgUGFwZXIgU2Npc3NvcnMgR2FtZSIpCnJvb3QuZ2VvbWV0cnko\nIjE5ODB4MTA4MCIpCnJvb3QuY29uZmlndXJlKGJnPSIjRjBGOEZGIikgICMg\nQWxpY2VCbHVlIGJhY2tncm91bmQKCiMgSGVhZGluZyBhdCB0aGUgdG9wIApo\nZWFkaW5nID0gdGsuTGFiZWwocm9vdCwgdGV4dD0iUm9jayBQYXBlciBTY2lz\nc29ycyBHYW1lIiwgZm9udD0oIkhlbHZldGljYSIsIDQwL

In [23]:
data['name']

'code_file.py'

### repo tree (not relevant?)

In [ ]:
# tree_sha = 'd1b07bc9e30f12188a6be5575fd4a27114cbf874'
# url_tree = f"https://api.github.com/repos/taniiishk/rock-paper-scissors-game/git/trees/{tree_sha}"

# payload = {}
# headers = {
#   'Accept': 'application/vnd.github+json',
#   'X-GitHub-Api-Version': '2022-11-28',
#   'Authorization': f'Bearer {ACCESS_TOKEN}'
# }

In [ ]:
# response = requests.request("GET", url_tree, headers=headers, data=payload)

In [ ]:
# response.json()

{'message': 'Invalid object requested. SHA must identify a commit or a tree.',
 'documentation_url': 'https://docs.github.com/rest/git/trees#get-a-tree',
 'status': '422'}